In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install arff
import arff

In [2]:
import pandas as pd
import numpy as np

**Парсил я в файле parser.py с использованием функций из parser_tools.py**

##стадии парсинга

In [ ]:
women = pd.read_csv('/content/drive/MyDrive/study/andan/women_perfume.csv', index_col=0)
men = pd.read_csv('/content/drive/MyDrive/study/andan/men_perfume.csv', index_col=0)
men['audience'] = 'men'
women['audience'] = 'women'

all = pd.concat([women, men], axis=0)

In [ ]:
all.rename(columns={"Год выпуска:": "year", "Группа ароматов:": "flavor_group", 'Верхние ноты:': 'upper_notes', 'Ноты сердца:': 'heart_notes',
                    'Базовые ноты:': 'base_notes', 'Страна производства:': 'country', 'Срок годности:': 'expiration', 'Производитель:': 'manufacturer'},
           inplace=True)

###унифицировать единицы измерения и очищать числовые значения от форматирования

In [ ]:
for i in range(all.shape[0]):
  line = all.iloc[i]
  volume = line['volume']
  price = line['price']
  if volume != '0' and isinstance(volume, str):
    volume_list_main = volume.split(' ')[0]
    if '+' in volume_list_main and '*' in volume_list_main:
      try:
        volume_list = volume_list_main.split('+')
        fin_volume = int(volume_list[0])
        volume_list2 = volume_list[1].split('*')
        fin_volume += int(volume_list2[0])
        fin_volume += int(volume_list2[1])
      except:
        volume_list = volume_list_main.split('*')
        fin_volume = int(volume_list[0])
        volume_list2 = volume_list[1].split('+')
        fin_volume += int(volume_list2[0])
        fin_volume += int(volume_list2[1])
    elif '+' in volume_list_main:
      volume_list = volume_list_main.split('+')
      fin_volume = 0
      for vol in volume_list:
        fin_volume += float(vol)
      volume_list_main = fin_volume
    elif '*' in volume_list_main:
      volume_list = volume_list_main.split('*')
      fin_volume = 1
      for vol in volume_list:
        fin_volume *= float(vol)
      volume_list_main = fin_volume
    all['volume'].replace(volume, volume_list_main, inplace=True)


  if price != '0' and isinstance(price, str):
    price_list = price.split(' ')
    new_pr = ''
    for pr in price_list:
      if pr != 'р.' and pr != ' ':
        new_pr += pr
    all['price'].replace(price, int(new_pr), inplace=True)

In [ ]:
all.replace('36 месяцев с момента апробирования', 36.0, inplace=True)
all['volume, ml'].replace('15+2*50', '115', inplace=True)
all.rename(columns={"expiration": "expiration, mounth", "volume": "volume, ml", 'price': 'price, rub'},
           inplace=True)

In [ ]:
all.dtypes

Сохранение в TSV

In [ ]:
all.to_csv('/content/drive/MyDrive/study/andan/perfume.tsv', sep='\t', index=False, header=True)

Сохранение в arff

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/study/andan/perfume.tsv',  sep='\t')
arff.dump(
    '/content/drive/MyDrive/study/andan/perfume.arff'
      , df.values
      , relation='perfume'
      , names=df.columns
)

##предобработка

In [10]:
df = pd.read_csv('/content/drive/MyDrive/study/andan/perfume.tsv',  sep='\t')

In [ ]:
a = list(df['flavor_group'].unique())
z = []
for i in g:
  if pd.notna(i):
    b = i.split(';')
    for j in b:
      z.append(j)
      if ('ые' not in j) and ('ие' not in j):
        print(j)

In [ ]:
def make_one_flavor_group_type(df):
    a = list(df['flavor_group'])
    z = []
    for i in a:
        if pd.notna(i):
            b = i.split(';')
            to_add = ''
            for j in b:
                if j == 'мускус':
                    to_add += 'мускусные;'
                elif j == 'древесина':
                    to_add += 'древесные;'
                elif j == 'альдегиды':
                    to_add += 'альдегидные;'
                elif j == 'ваниль':
                    to_add += 'ванильные;'
                else:
                    to_add += j + ';'
            z.append(to_add)
        else:
            z.append(np.nan)
    return z

In [ ]:
df.shape

(28286, 13)

In [ ]:
# парсинг делался по компании и названию, следовательно эти строки полностью фиктивны
df.drop(df[df['company'].isnull()].index, inplace=True)
df.drop(df[df['name'].isnull()].index, inplace=True)
# если неизвестно чем пахнут, то не на чем предсказывать
df.drop(df[df['flavor_group'].isnull()].index, inplace=True)
# сновные фичи для предсказания, если нул, то строки фиктивны
df.drop(df[df['upper_notes'].isnull() & df['heart_notes'].isnull() & df['base_notes'].isnull()].index, inplace=True)

In [ ]:
# заполняю год и срок хранения средним
df['year'].fillna((df['year'].mean()), inplace=True)
df['expiration, mounth'].fillna((df['expiration, mounth'].mean()), inplace=True)

# При отсутствии места производства и страны говорю что его "нет"
df['manufacturer'].fillna('no', inplace=True)
df['country'].fillna('no', inplace=True)

In [ ]:
df.columns

Index(['company', 'name', 'year', 'flavor_group', 'upper_notes', 'heart_notes',
       'base_notes', 'country', 'expiration, mounth', 'volume, ml',
       'price, rub', 'manufacturer', 'audience'],
      dtype='object')

Дальнейшая обработка:

**audience** - целевой признак.

Существуют такие колонки как 'flavor_group', 'upper_notes', 'heart_notes', 'base_notes'.

flavor_group - группа запахов, обощенная инфа чем пахнет в среднем.

upper_notes - верхние ноты, то что чувствует неопытный человек, самые явные запахи.

heart_notes - тонкие запахи, пахнут слабее но задают явную картину.

base_notes - фоновые ноты, придают плотность, держатся дольше.

flavor_group я разделяю на колонки и задаю 0 или 1 в зависимости от наличия.



Для остальных - так как у них имеется условная "мощность", и в основном пересекающиеся значения, то обработка идет следующим образом:
1. Все колонки объеденяются в общий лист запахов
2. В зависимости от включения запахов в фичу, каждому семплу назначается вектор состоящий из чисел (числа обсуждаемы):


      0 - отсутствие

      1 - base_notes

      2 - heart_notes

      3 - upper_notes


Это основная информация, на которой планируется строить модель.

Такие колонки как 'company', 'name', 'country', 'manufacturer' - несут мало ценной информации в рамках идеи, пойдут как сопутствующая информация.

'price, rub', 'volume, ml', 'year' - возможны в использование, но будут обрабатываться отдельно (например, не будут включены в операцию по снижению размерности и тому подобное).

'price, rub'/'volume, ml' - потенциальный таргет для регрессии (цена одного миллилитра).

'expiration, mounth' - везде одинаков, не представляет интересной информации.

In [ ]:
def make_list_flavor(df, col_name):
    complex_flavor_group = list(df[col_name].unique())
    all_flavor_group = []
    for complex_flavor in complex_flavor_group:
        if pd.notna(complex_flavor):
            flavors = complex_flavor.split(';')
            for flavor in flavors:
                if flavor not in all_flavor_group:
                    all_flavor_group.append(flavor)
    return all_flavor_group


def extract_columns(df, col_name):
    all_flavor_group = make_list_flavor(df, col_name)
    print(all_flavor_group)
    if 'iso e super' in all_flavor_group:
        print(col_name)
        print(all_flavor_group)
        all_flavor_group.remove('iso e super')

    all_flavor_group_dict_cols = {}
    df_col_flavor_group = df[col_name]
    for flavor in all_flavor_group:
        df_col_list = []
        for df_col in df_col_flavor_group:
            if pd.notna(df_col):
                if flavor in df_col:
                    df_col_list.append(1)
                else:
                    df_col_list.append(0)
            else:
                df_col_list.append(0)
        all_flavor_group_dict_cols[flavor] = df_col_list
    return all_flavor_group_dict_cols


def add_flav_columns(df, col_name):
    flavor_group_columns = extract_columns(df, col_name)
    for i in flavor_group_columns.keys():
        df[i + '_f_g'] = flavor_group_columns[i]


def make_one_flavor_group_type(df):
    a = list(df['flavor_group'])
    z = []
    for i in a:
        if pd.notna(i):
            b = i.split(';')
            to_add = ''
            for j in b:
                if j == 'мускус':
                    to_add += 'мускусные;'
                elif j == 'древесина':
                    to_add += 'древесные;'
                elif j == 'альдегиды':
                    to_add += 'альдегидные;'
                elif j == 'ваниль':
                    to_add += 'ванильные;'
                else:
                    to_add += j + ';'
            z.append(to_add)
        else:
            z.append(np.nan)
    return z


In [ ]:
df['flavor_group'] = make_one_flavor_group_type(df)
add_flav_columns(df, 'flavor_group')

complex_upper_notes = make_list_flavor(df, 'upper_notes')
complex_heart_notes = make_list_flavor(df, 'heart_notes')
complex_base_notes = make_list_flavor(df, 'base_notes')

all_notes = list(set(complex_upper_notes+complex_heart_notes+complex_base_notes))


# Следующие вычисления очень долгие +-час, в зависимости от машины

all_flavor_group_dict_cols = {}
for flavor in all_notes:
    df_col_list = []
    for _id in range(df.shape[0]):
        line = df.iloc[_id]
        upper_note = line['upper_notes']
        heart_note = line['heart_notes']
        base_note = line['base_notes']
        if pd.notna(upper_note) and flavor in upper_note:
            df_col_list.append(3)
        elif pd.notna(heart_note) and flavor in heart_note:
            df_col_list.append(2)
        elif pd.notna(base_note) and flavor in base_note:
            df_col_list.append(1)
        else:
            df_col_list.append(0)

    all_flavor_group_dict_cols[flavor] = df_col_list

In [ ]:
all_flavor_group_df = pd.DataFrame.from_dict(all_flavor_group_dict_cols)

df = pd.concat([df, all_flavor_group_df], axis=1)

In [ ]:
df.drop(['upper_notes', 'heart_notes', 'base_notes', 'flavor_group'], axis=1, inplace=True)

In [ ]:
df.to_csv('/content/drive/MyDrive/study/andan/perfume_processed.csv')

In [23]:
df_processed = pd.read_csv('/content/drive/MyDrive/study/andan/perfume_processed.csv')

In [24]:
df_processed.drop(['_f_g', 'expiration, mounth'], axis=1, inplace=True)
df_processed.rename(columns={"Unnamed: 0": "ID"}, inplace=True)

df_additional = df_processed[['ID', 'price, rub', 'volume, ml', 'year']]
df_support_info = df_processed[['ID', 'company', 'name', 'country', 'manufacturer']]

df_target = df_processed[['ID', 'audience']]
df_target_id = df_target.replace('women', 1)
df_target_id = df_target_id.replace('men', 2)
df_target['class'] = df_target_id['audience']
df_main = df_processed.drop(['price, rub', 'volume, ml', 'year', 'company', 'name', 'country', 'manufacturer', 'audience'], axis=1)

<ipython-input-24-33ce2a3c1ad4>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_target_id = df_target_id.replace('men', 2)
<ipython-input-24-33ce2a3c1ad4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['class'] = df_target_id['audience']


In [25]:
# минмакс
def normalize(x):
    return [round((i - min(x)) / (max(x) - min(x)), 2) for i in x]

# моя крутая нормализация
df_to_norm = df_main.drop(['ID'], axis=1)
df_to_norm.replace(1, 0.33, inplace=True)
df_to_norm.replace(2, 0.67, inplace=True)
df_to_norm.replace(3, 1.0, inplace=True)
df_to_norm['ID'] = df_main['ID']
df_to_norm.shape

<ipython-input-25-a83897ad22d3>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_to_norm['ID'] = df_main['ID']


(27192, 1090)

In [26]:
df_to_norm.set_index('ID', inplace=True)
df_target.set_index('ID', inplace=True)
df_support_info.set_index('ID', inplace=True)
df_additional.set_index('ID', inplace=True)

In [27]:
df_to_norm.drop_duplicates(inplace=True)

In [28]:
df_target = df_target.loc[df_to_norm.index]
df_additional = df_additional.loc[df_to_norm.index]
df_support_info = df_support_info.loc[df_to_norm.index]

In [29]:
df_to_norm.reset_index(inplace=True, drop=True)
df_target.reset_index(inplace=True, drop=True)
df_support_info.reset_index(inplace=True, drop=True)
df_additional.reset_index(inplace=True, drop=True)

In [30]:
df_to_norm

,восточные_f_g,цветочные_f_g,древесные_f_g,фужерные_f_g,водяные_f_g,шипровые_f_g,гурманские_f_g,мускусные_f_g,кожаные_f_g,зеленые_f_g,...,чай эрл грей,капучино,тысячелистник,пеларгония,белый табак,скотч,кринум,цедра мандарина,камыш,полынь
0,0.33,0.33,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.33,0.33,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.33,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.00,0.33,0.33,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.33,0.00,0.33,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26640,0.00,0.33,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26641,0.33,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26642,0.33,0.00,0.00,0.33,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26643,0.00,0.33,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df_to_norm.to_csv('/content/drive/MyDrive/study/andan/df_main.csv')
df_target.to_csv('/content/drive/MyDrive/study/andan/df_target.csv')
df_support_info.to_csv('/content/drive/MyDrive/study/andan/df_support_info.csv')
df_additional.to_csv('/content/drive/MyDrive/study/andan/df_additional.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/study/andan/perfume.tsv',  sep='\t')
df_processed = pd.read_csv('/content/drive/MyDrive/study/andan/perfume_processed.csv')

In [ ]:
df.iloc[1]['upper_notes']

'бергамот;кориандр;корица;мандарин;нероли;палисандр;лист томата'

In [ ]:
df_processed.iloc[1]['мандарин']

In [ ]:
df1 = df_processed[df_processed.isna().any(axis=1)]
df1

,Unnamed: 0,company,name,year,country,"expiration, mounth","volume, ml","price, rub",manufacturer,audience,...,чай эрл грей,капучино,тысячелистник,пеларгония,белый табак,скотч,кринум,цедра мандарина,камыш,полынь
